In [352]:
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [353]:
pip install plotly

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [354]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [355]:
pip install seaborn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [356]:
pip install dash

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [357]:
# Import standard libraries
import os
from contextlib import redirect_stdout

import sys
# append coeqwal packages to path
sys.path.append('./coeqwalpackage')

import numpy as np
import pandas as pd
import datetime as dt
import re
import plotly.graph_objects as go
import dash
from dash import html, dcc
from dash.dependencies import Input, Output

In [358]:
# Import custom libraries
# Note: on my computer the next import doesn't work the first time I call it, why? If I re-run the cell, then it is ok. MUST DEBUG
from coeqwalpackage.metrics import *
import cqwlutils as cu
import plotting as pu

## Interactive Dashboard

### Time Series
Shows how the selected variable changes over time for each scenario.

### Monthly-of-Year
Displays the monthly average for a selected year.

### Single Exceedance
Shows the probability that a value will be equaled or exceeded.

### Annual Exceedance
Shows how often the selected month's total value exceeds a given threshold across all years. Each year’s data for the chosen month is summed (e.g., total flow in April each year), and the annual values are ranked from highest to lowest. From these ranks, exceedance probabilities are calculated to show how frequently high values occur.

### Month-of-Year Avg
Averages each calendar month across all years, optionally filtered by Water Year Type. Water Year Types classify each year based on how wet or dry it was. The scale ranges from 1 (wettest) to 5 (driest)

In [359]:
CtrlFile = 'CalSim3DataExtractionInitFile_v4.xlsx'
CtrlTab = 'Init'
ScenarioListFile, ScenarioListTab, ScenarioListPath, DVDssNamesOutPath, SVDssNamesOutPath, ScenarioIndicesOutPath, DssDirsOutPath, VarListPath, VarListFile, VarListTab, VarOutPath, DataOutPath, ConvertDataOutPath, ExtractionSubPath, DemandDeliverySubPath, ModelSubPath, GroupDataDirPath, ScenarioDir, DVDssMin, DVDssMax, SVDssMin, SVDssMax, NameMin, NameMax, DirMin, DirMax, IndexMin, IndexMax, StartMin, StartMax, EndMin, EndMax, VarMin, VarMax, DemandFilePath, DemandFileName, DemandFileTab, DemMin, DemMax, InflowOutSubPath, InflowFilePath, InflowFileName, InflowFileTab, InflowMin, InflowMax = cu.read_init_file(CtrlFile, CtrlTab)

In [360]:
df, dss_names = read_in_df(ConvertDataOutPath,DVDssNamesOutPath)

In [361]:
nMin = int(re.search(r'\d+', VarMin).group())
nMax = int(re.search(r'\d+', VarMax).group())
nRows = nMax - nMin + 1
metadata_df, _ = load_metadata_df(ScenarioDir + '/', DataOutPath, VarListFile, nrows=nRows)

# Safety: df must exist & have DatetimeIndex
if 'df' not in globals():
    raise RuntimeError("Please load your DataFrame into variable `df` (DatetimeIndex required) before running.")
if not isinstance(df.index, pd.DatetimeIndex):
    df.index = pd.to_datetime(df.index)

In [362]:
def _extract_var_from_mi_level_B(level_B: str) -> str:
    m = re.match(r"^(.*?)_s\d{4}$", str(level_B))
    return m.group(1) if m else ""

def _extract_var_from_flat(name: str) -> str:
    m = re.search(r"^CALSIM_(.*?)_s\d{4}", str(name))
    return m.group(1) if m else ""

def _match_target_var(v: str) -> bool:
    return v.startswith("C_") or v == "NDO" or v.startswith("NDOI_")

def columns_to_convert(df_in: pd.DataFrame):
    if isinstance(df_in.columns, pd.MultiIndex):
        return [
            col for col in df_in.columns
            if str(col[-1]).upper() == "CFS"
            and _match_target_var(_extract_var_from_mi_level_B(col[1]) if len(col) > 1 else "")
        ]
    else:
        return [
            c for c in df_in.columns
            if str(c).endswith("_CFS")
            and _match_target_var(_extract_var_from_flat(str(c)))
        ]


In [363]:
def apply_package_convert_selected(df_in: pd.DataFrame, metadata_df: pd.DataFrame) -> pd.DataFrame:
    """
    Use ONLY the package convert_cfs_to_taf on selected columns:
      - Variables: C_*, NDO, NDOI_*
      - Unit: CFS
    Merge back and drop CFS twins where a TAF exists.
    """
    if isinstance(df_in.columns, pd.MultiIndex):
        target_cols = columns_to_convert(df_in)
        if not target_cols:
            return df_in.copy()
        sub = df_in.loc[:, target_cols].copy()
        sub_after = convert_cfs_to_taf(sub, metadata_df)
        if sub_after is None:
            sub_after = sub

        out = df_in.copy()
        for col in sub_after.columns:
            out[col] = sub_after[col]

        # drop CFS twins
        cols_set = set(out.columns)
        to_drop = [c for c in out.columns if (str(c[-1]).upper() == "CFS") and (c[:-1] + ("TAF",)) in cols_set]
        if to_drop:
            out = out.drop(columns=to_drop)
        return out

    else:
        target_cols = columns_to_convert(df_in)
        if not target_cols:
            return df_in.copy()
        sub = df_in.loc[:, target_cols].copy()
        sub_after = convert_cfs_to_taf(sub, metadata_df)
        if sub_after is None:
            sub_after = sub

        out = df_in.copy()
        for col in sub_after.columns:
            out[col] = sub_after[col]

        # drop CFS twins
        colset = set(out.columns)
        to_drop = [c for c in out.columns if str(c).endswith("_CFS") and (str(c)[:-4] + "TAF") in colset]
        if to_drop:
            out = out.drop(columns=to_drop)
        return out


In [364]:
# Convert only target vars using the package
df = apply_package_convert_selected(df, metadata_df)

# Flatten columns for the dashboard (if needed)
if isinstance(df.columns, pd.MultiIndex):
    df.columns = ['_'.join(map(str, col)) for col in df.columns]



Units Mapping:
C_TRNTY: TAF
C_LWSTN: TAF
D_LWSTN_CCT011: TAF
C_WKYTN: TAF
C_SHSTA: TAF
C_KSWCK_MIF: TAF
C_KSWCK: TAF
C_SAC257: TAF
C_SAC257_MIF: TAF
C_SAC257_FLOW_SELECTDV: TAF
D_SAC240_TCC001: TAF
C_SAC240: TAF
D_SAC207_GCC007: TAF
C_SAC201: TAF
C_STN026: TAF
C_SAC120: TAF
C_OROVL: TAF
C_FTR068: TAF
C_FTR059_MIF: TAF
C_FTR059: TAF
C_YUB002: TAF
C_YUB002_MIF: TAF
C_YUB002_FLOW_SELECTDV: TAF
C_FTR028: TAF
C_FTR012: TAF
C_FTR003: TAF
C_FTR003_MIF: TAF
C_FTR003_FLOW_SELECTDV: TAF
C_FOLSM: TAF
C_NTOMA_MIF: TAF
C_NTOMA: TAF
C_AMR004: TAF
C_AMR004_MIF: TAF
C_AMR004_FLOW_SELECTDV: TAF
SP_SAC083_YBP037: TAF
C_SAC083: TAF
C_SAC048: TAF
C_SAC041: TAF
C_YBP020: TAF
C_MOK019: TAF
C_CLV004: TAF
C_MELON: TAF
C_STS059_MIF: TAF
C_STS059: TAF
C_STS004: TAF
C_TUO003: TAF
C_TUO003_MIF: TAF
C_TUO003_FLOW_SELECTDV: TAF
C_MCD021: TAF
C_CHW017: TAF
C_EBP016: TAF
D_MLRTN_FRK000: TAF
D_MLRTN_MDC006: TAF
C_SJR225: TAF
C_SJR180: TAF
C_SJR115: TAF
C_SJR070: TAF
C_SJR070_MIF: TAF
C_SJR070_FLOW_SELECTDV: TAF
D_SAC

/Users/rain/Desktop/DSP/coeqwal/notebooks/coeqwalpackage/metrics.py:158: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/Users/rain/Desktop/DSP/coeqwal/notebooks/coeqwalpackage/metrics.py:158: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/Users/rain/Desktop/DSP/coeqwal/notebooks/coeqwalpackage/metrics.py:158: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To ge

Updated column units to 'TAF' for ('CALSIM', 'C_FTR068_s0005', 'CHANNEL', '1MON', 'L2020A', 'PER-AVER', 'TAF')

Converting column: ('CALSIM', 'C_KSWCK_s0005', 'CHANNEL', '1MON', 'L2020A', 'PER-AVER', 'CFS') from CFS to TAF
Updated column units to 'TAF' for ('CALSIM', 'C_KSWCK_s0005', 'CHANNEL', '1MON', 'L2020A', 'PER-AVER', 'TAF')

Converting column: ('CALSIM', 'C_KSWCK_ADD_s0005', 'FLOW-ADDITIONAL-INSTREAM', '1MON', 'L2020A', 'PER-AVER', 'CFS') from CFS to TAF
Updated column units to 'TAF' for ('CALSIM', 'C_KSWCK_ADD_s0005', 'FLOW-ADDITIONAL-INSTREAM', '1MON', 'L2020A', 'PER-AVER', 'TAF')

Converting column: ('CALSIM', 'C_KSWCK_MIF_s0005', 'FLOW-MIN-INSTREAM', '1MON', 'L2020A', 'PER-AVER', 'CFS') from CFS to TAF
Updated column units to 'TAF' for ('CALSIM', 'C_KSWCK_MIF_s0005', 'FLOW-MIN-INSTREAM', '1MON', 'L2020A', 'PER-AVER', 'TAF')

Converting column: ('CALSIM', 'C_LWSTN_s0005', 'CHANNEL', '1MON', 'L2020A', 'PER-AVER', 'CFS') from CFS to TAF
Updated column units to 'TAF' for ('CALSI

/Users/rain/Desktop/DSP/coeqwal/notebooks/coeqwalpackage/metrics.py:158: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/Users/rain/Desktop/DSP/coeqwal/notebooks/coeqwalpackage/metrics.py:158: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/Users/rain/Desktop/DSP/coeqwal/notebooks/coeqwalpackage/metrics.py:158: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To ge

Updated column units to 'TAF' for ('CALSIM', 'C_STS011_s0011', 'CHANNEL', '1MON', 'L2020A', 'PER-AVER', 'TAF')

Converting column: ('CALSIM', 'C_STS011_MIF_s0011', 'FLOW-MIN-INSTREAM', '1MON', 'L2020A', 'PER-AVER', 'CFS') from CFS to TAF
Updated column units to 'TAF' for ('CALSIM', 'C_STS011_MIF_s0011', 'FLOW-MIN-INSTREAM', '1MON', 'L2020A', 'PER-AVER', 'TAF')

Converting column: ('CALSIM', 'C_STS017_s0011', 'CHANNEL', '1MON', 'L2020A', 'PER-AVER', 'CFS') from CFS to TAF
Updated column units to 'TAF' for ('CALSIM', 'C_STS017_s0011', 'CHANNEL', '1MON', 'L2020A', 'PER-AVER', 'TAF')

Converting column: ('CALSIM', 'C_STS059_s0011', 'CHANNEL', '1MON', 'L2020A', 'PER-AVER', 'CFS') from CFS to TAF
Updated column units to 'TAF' for ('CALSIM', 'C_STS059_s0011', 'CHANNEL', '1MON', 'L2020A', 'PER-AVER', 'TAF')

Converting column: ('CALSIM', 'C_STS059_MIF_s0011', 'FLOW-MIN-INSTREAM', '1MON', 'L2020A', 'PER-AVER', 'CFS') from CFS to TAF
Updated column units to 'TAF' for ('CALSIM', 'C_STS059_MIF_s00

/Users/rain/Desktop/DSP/coeqwal/notebooks/coeqwalpackage/metrics.py:158: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/Users/rain/Desktop/DSP/coeqwal/notebooks/coeqwalpackage/metrics.py:158: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/Users/rain/Desktop/DSP/coeqwal/notebooks/coeqwalpackage/metrics.py:158: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To ge

In [365]:
original_columns = df.columns.tolist()

# scenarios like s0001, s0018
s_numbers = set()
for col in original_columns:
    s_numbers.update(re.findall(r's\d{4,}', col))
scenarios = sorted(s_numbers)
default_scenario = [scenarios[0]] if scenarios else []

# Map var -> unit; prefer TAF if mixed
variable_units = {}
variables = set()
pattern_var = re.compile(r'CALSIM_(.*?)_s\d{4}', re.IGNORECASE)

for col in df.columns:
    m = pattern_var.search(col)
    if m:
        var_name = m.group(1)
        unit = col.split('_')[-1].upper()
        variables.add(var_name)
        if var_name not in variable_units or unit == "TAF":
            variable_units[var_name] = unit

variables = sorted(variables)
variable_labels = [{"label": f"{v} ({variable_units.get(v, 'UNKNOWN')})", "value": v} for v in variables]


In [366]:
def add_water_year_column(df_in):
    d = df_in.copy().sort_index()
    d['Date'] = pd.to_datetime(d.index)
    d['Year'] = d['Date'].dt.year
    d['Month'] = d['Date'].dt.month
    d['WaterYear'] = np.where(d['Month'] >= 10, d['Year'] + 1, d['Year'])
    return d.drop(["Date", "Year", "Month"], axis=1)

water_year_df = add_water_year_column(df)

years_in_data = sorted(pd.DatetimeIndex(df.index).year.unique())
drought_years = {1924, 1925, 1926, 1929, 1930, 1931, 1932, 1933, 1934, 1939,
                  1944, 1945, 1947, 1948, 1949, 1950, 1955, 1960, 1961, 1962, 1964,
                  1976, 1977, 1979, 1981, 1987, 1988, 1989, 1990, 1991, 1992, 1994,
                  2001, 2008, 2009, 2013, 2014, 2015, 2020, 2021}
year_options = [{"label": f"{y} {'(Drought)' if y in drought_years else ''}", "value": y} for y in years_in_data]
water_year_type_options = [{"label": str(i), "value": i} for i in range(1, 6)]
month_options = [{"label": "April", "value": 4}, {"label": "September", "value": 9}]

plot_type_descriptions = {
    "time_series": "Shows how the selected variable changes over time for each scenario.",
    "monthly": "Displays the monthly average for a selected year.",
    "single_exceedance": "Shows the probability that a value will be equaled or exceeded.",
    "annual_exceedance": "Shows how often the selected month's total value exceeds a given threshold across all years.",
    "month_of_year_avg": "Averages each calendar month across all years, optionally filtered by Water Year Type."
}


In [367]:
def find_col(df_in, variable, scenario):
    candidates = [c for c in df_in.columns if f"CALSIM_{variable}_" in c and scenario in c]
    if not candidates:
        candidates = [c for c in df_in.columns if variable in c and scenario in c]
    candidates.sort(key=lambda x: (not x.endswith("_TAF"), not x.endswith("_CFS"), x))
    return candidates[0] if candidates else None

def find_wyt_col(df_in, scenario):
    matches = [col for col in df_in.columns if f"CALSIM_WYT_SAC__{scenario}" in col and "WATERYEARTYPE" in col]
    return matches[0] if matches else None

def get_colors(scnrs):
    base = ["red", "blue", "green", "orange", "purple", "brown", "cyan", "magenta", "gray", "black"]
    return {s: base[i % len(base)] for i, s in enumerate(scnrs)}

def get_line_styles():
    return ['solid', 'dash', 'dot', 'dashdot', 'longdash', 'longdashdot']

def filter_by_wyt_annual(df_col, scenario, wyt_list, month=5):
    if not wyt_list:
        return df_col
    wyt_col = find_wyt_col(df, scenario)
    if wyt_col is None:
        return df_col
    working_df = df[[wyt_col]].copy()
    working_df['WaterYear'] = water_year_df['WaterYear']
    working_df['Month'] = pd.DatetimeIndex(df.index).month
    filtered = working_df[working_df['Month'] == month].groupby('WaterYear').first()
    selected_years = filtered[filtered[wyt_col].isin(wyt_list)].index
    df_out = df_col.copy()
    df_out['WaterYear'] = water_year_df['WaterYear']
    df_out = df_out[df_out['WaterYear'].isin(selected_years)]
    return df_out.drop(columns='WaterYear')


In [368]:
app = dash.Dash(__name__)
app.layout = html.Div([
    html.H2("Water Data Dashboard", style={'textAlign': 'center'}),
    html.Label("Select Variables (units reflect conversion)"),
    dcc.Dropdown(id="variable-dropdown", options=variable_labels,
                 value=[variables[0]] if variables else [], multi=True),

    html.Label("Select Scenarios"),
    dcc.Dropdown(id="scenario-dropdown", options=[{"label": s, "value": s} for s in scenarios],
                 value=[scenarios[0]] if scenarios else [], multi=True),

    html.Label("Select Plot Type"),
    dcc.Dropdown(id="plot-type-dropdown", options=[
        {"label": "Time Series", "value": "time_series"},
        {"label": "Monthly-of-Year", "value": "monthly"},
        {"label": "Month-of-Year Avg", "value": "month_of_year_avg"},
        {"label": "Single Exceedance", "value": "single_exceedance"},
        {"label": "Annual Exceedance", "value": "annual_exceedance"}
    ], value="time_series"),

    html.Div(id="plot-type-description", style={"marginTop": "10px", "fontStyle": "italic", "color": "#555"}),

    html.Div([
        html.Label("Select Year (for Monthly Plot)"),
        dcc.Dropdown(id="year-dropdown", options=year_options, value=years_in_data[0] if years_in_data else None)
    ], id="year-container", style={"display": "none"}),

    html.Div([
        html.Label("Select Water Year Type (1-5)"),
        dcc.Dropdown(id="wyt-dropdown", options=water_year_type_options, value=[], multi=True)
    ], id="wyt-container", style={"display": "none"}),

    html.Div([
        html.Label("Select Month for Annual Exceedance"),
        dcc.Dropdown(id="month-dropdown", options=month_options, value=9)
    ], id="month-container", style={"display": "none"}),

    dcc.Graph(id="dynamic-plot")
])

if __name__ == '__main__':
    app.run(debug=True)

@app.callback(
    [Output("dynamic-plot", "figure"),
     Output("year-container", "style"),
     Output("wyt-container", "style"),
     Output("month-container", "style"),
     Output("plot-type-description", "children")],
    [Input("variable-dropdown", "value"),
     Input("scenario-dropdown", "value"),
     Input("plot-type-dropdown", "value"),
     Input("year-dropdown", "value"),
     Input("wyt-dropdown", "value"),
     Input("month-dropdown", "value")]
)
def update_plot(vars_selected, scenarios_sel, plot_type, year, wyt, month):
    show_year = {"display": "block"} if plot_type == "monthly" else {"display": "none"}
    show_wyt = {"display": "block"} if plot_type == "month_of_year_avg" else {"display": "none"}
    show_month = {"display": "block"} if plot_type == "annual_exceedance" else {"display": "none"}
    description = plot_type_descriptions.get(plot_type, "")

    if not scenarios_sel or not vars_selected:
        return go.Figure(), show_year, show_wyt, show_month, description

    fig = go.Figure()
    line_styles = get_line_styles()
    colors = get_colors(scenarios_sel)

    units_used = set()  # derive from columns actually plotted

    for idx, v in enumerate(vars_selected):
        style = line_styles[idx % len(line_styles)]
        for s in scenarios_sel:
            col = find_col(df, v, s)
            if not col:
                continue

            this_unit = col.split('_')[-1].upper()
            units_used.add(this_unit)

            df_copy = df[[col]].copy()

            if plot_type == "monthly":
                df_copy['Year'] = pd.DatetimeIndex(df_copy.index).year
                df_copy['Month'] = pd.DatetimeIndex(df_copy.index).month
                df_copy = df_copy[df_copy['Year'] == year]
                monthly_avg = df_copy.groupby('Month')[col].mean()
                fig.add_trace(go.Scatter(
                    x=monthly_avg.index, y=monthly_avg.values,
                    mode='lines+markers', name=f"{s} - {v}",
                    line=dict(color=colors[s], dash=style)
                ))

            elif plot_type == "month_of_year_avg":
                tmp = water_year_df[[col]].copy()
                tmp = filter_by_wyt_annual(tmp, s, wyt, month=5)
                tmp['Month'] = pd.DatetimeIndex(tmp.index).month
                monthly_avg = tmp.groupby('Month')[col].mean()
                fig.add_trace(go.Scatter(
                    x=monthly_avg.index, y=monthly_avg.values,
                    mode='lines', name=f"{s} - {v}",
                    line=dict(color=colors[s], dash=style)
                ))

            elif plot_type == "single_exceedance":
                series = df_copy[col].dropna().sort_values(ascending=False)
                exceedance_probs = np.arange(1, len(series)+1) / (len(series)+1)
                fig.add_trace(go.Scatter(
                    x=exceedance_probs, y=series.values,
                    mode='lines', name=f"{s} - {v}",
                    line=dict(color=colors[s], dash=style)
                ))

            elif plot_type == "annual_exceedance":
                df_mo = df_copy[pd.DatetimeIndex(df_copy.index).month == month]
                annual_sum = df_mo.resample("YE").sum(min_count=1)
                sorted_vals = annual_sum[col].dropna().sort_values(ascending=False)
                exceed_probs = sorted_vals.rank(method="first", ascending=False) / (1 + len(sorted_vals))
                fig.add_trace(go.Scatter(
                    x=exceed_probs, y=sorted_vals,
                    mode='lines', name=f"{s} - {v}",
                    line=dict(color=colors[s], dash=style)
                ))

            else:  # time_series
                fig.add_trace(go.Scatter(
                    x=df.index, y=df_copy[col],
                    mode='lines', name=f"{s} - {v}",
                    line=dict(color=colors[s], dash=style)
                ))

    # y-axis from what we actually plotted
    if not units_used:
        y_label = "Value"
    elif len(units_used) == 1:
        y_label = f"Value ({next(iter(units_used))})"
    else:
        y_label = f"Value (Mixed: {', '.join(sorted(units_used))})"

    fig.update_layout(
        title=f"{plot_type.replace('_', ' ').title()} Plot for Selected Variables",
        xaxis_title="Date" if plot_type in ["time_series", "monthly", "month_of_year_avg"] else "Exceedance Probability",
        yaxis_title=y_label
    )
    return fig, show_year, show_wyt, show_month, description

